In [ ]:
##%pip install torchvision

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ------------------------------------ --- 1.6/1.7 MB 12.9 MB/s eta 0:00:01
   ---------------------------------------- 1.7/1.7 MB 12.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# Set random seed for reproducibility
torch.manual_seed(4053)
np.random.seed(4053)
if torch.cuda.is_available():
    torch.cuda.manual_seed(4053)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [ ]:
# 1. Data Loading and Preprocessing
# CIFAR-10 dataset has images of size 3x32x32 (color, 32x32 pixels)
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 64 # You can adjust this

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')


In [ ]:
# 2. Define the CNN Model
class BasicCNN(nn.Module):
    def __init__(self):
        super(BasicCNN, self).__init__()
        # First Convolutional Layer
        # Input: 3x32x32 (channels, height, width)
        # Output after conv1: (32-3+2*1)/1 + 1 = 32 -> out_channels x 32 x 32
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        self.relu1 = nn.ReLU()
        # Max Pooling Layer
        # Output after pool1: 32x16x16 (max_pool2d with kernel_size=2 reduces H, W by half)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Second Convolutional Layer
        # Input: 32x16x16
        # Output after conv2: (16-3+2*1)/1 + 1 = 16 -> out_channels x 16 x 16
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.relu2 = nn.ReLU()
        # Output after pool2: 64x8x8

        # Fully Connected Layer
        # Flatten the output from conv layers: 64 * 8 * 8 = 4096
        self.fc1 = nn.Linear(64 * 8 * 8, 128) # You can adjust hidden layer size
        self.relu3 = nn.ReLU()
        # Output layer for 10 classes
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(self.relu1(self.conv1(x)))
        x = self.pool(self.relu2(self.conv2(x)))
        x = x.view(-1, 64 * 8 * 8) # Flatten the tensor for the fully connected layer
        x = self.relu3(self.fc1(x))
        x = self.fc2(x)
        return x

# Initialize the model, loss function, and optimizer
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

net = BasicCNN().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
# 3. Training the Model
total_epochs = 10
train_loss_history = []
train_accuracy_history = []
val_loss_history = []
val_accuracy_history = []

print("Starting Training...")
for epoch in range(total_epochs):  # loop over the dataset multiple times
    net.train() # Set the model to training mode
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        _, predicted = torch.max(outputs.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    train_loss = running_loss / len(trainloader)
    train_accuracy = 100 * correct_train / total_train
    train_loss_history.append(train_loss)
    train_accuracy_history.append(train_accuracy)

    # Validation phase
    net.eval() # Set the model to evaluation mode
    val_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad(): # No gradient calculation during validation
        for data in testloader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = net(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_loss = val_loss / len(testloader)
    val_accuracy = 100 * correct_val / total_val
    val_loss_history.append(val_loss)
    val_accuracy_history.append(val_accuracy)

    print(f'Epoch {epoch + 1}/{total_epochs}, '
          f'Train Loss: {train_loss:.3f}, Train Acc: {train_accuracy:.2f}%, '
          f'Val Loss: {val_loss:.3f}, Val Acc: {val_accuracy:.2f}%')

print('Finished Training')

In [ ]:
# 4. Plotting Training and Validation Loss/Accuracy
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(range(1, total_epochs + 1), train_loss_history, label='Train Loss')
plt.plot(range(1, total_epochs + 1), val_loss_history, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(range(1, total_epochs + 1), train_accuracy_history, label='Train Accuracy')
plt.plot(range(1, total_epochs + 1), val_accuracy_history, label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy (%)')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()


In [ ]:
# 5. Evaluate on the entire test set (final check)
print("\nEvaluating on test set...")
net.eval() # Set the model to evaluation mode
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct / total:.2f}%')

# Optional: Per-class accuracy
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(len(labels)):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1

print("\nAccuracy per class:")
for i in range(10):
    print(f'Accuracy of {classes[i]:5s} : {100 * class_correct[i] / class_total[i]:.2f}%')